# Проект-ДЗ по Эксплуатации

В данном проекте вы выполните комплексное домашнее задание по подготовке, оптимизации и развертыванию модели машинного обучения с использованием современных инструментов. Цель проекта – освоить процессы обучения, конвертации, оптимизации и интеграции моделей в продакшн-среду с применением Triton Inference Server, Docker и микросервисной архитектуры.

In [1]:
! python -m venv .venv

1. **Обучение модели**
   - Обучите самописную модель на базе **Torch** или **TensorFlow**.
   - Используйте стандартные слои, избегая кастомных решений.

In [1]:
! pip install torch torchvision

  Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
  Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


You should consider upgrading via the 'F:\GitHub\HSE\dev_ops\hw1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Среднее и стандартное отклонение MNIST
])

train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU() 
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.fc = nn.Linear(64 * 28 * 28, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = x.view(x.shape[0], -1)  # Flatten
        x = self.fc(x)
        return x

In [4]:
! nvidia-smi

Sun Mar 16 17:35:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.70                 Driver Version: 572.70         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   36C    P8             35W /  320W |    1607MiB /  10240MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
! pip uninstall torch torchvision torchaudio --yes

In [4]:
! -m pip install --upgrade pip

"-m" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [7]:
! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.6.0%2Bcu126-cp310-cp310-win_amd64.whl (2496.1 MB)


You should consider upgrading via the 'F:\GitHub\HSE\dev_ops\hw1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

device

device(type='cuda')

In [12]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

In [13]:
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)  # Получаем индекс максимального логита
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')

In [6]:
NUM_EPOCHS = 5
MODEL_NAME = "mnist_cnn.pth"

In [ ]:
for epoch in range(1, NUM_EPOCHS + 1):
    train(model, train_loader, optimizer, criterion, epoch)
    test(model, test_loader, criterion)

torch.save(model.state_dict(), MODEL_NAME)
print(f"Модель сохранена в {MODEL_NAME}")

Train Epoch: 1 [0/60000] Loss: 2.312083
Train Epoch: 1 [6400/60000] Loss: 0.140889
Train Epoch: 1 [12800/60000] Loss: 0.091742
Train Epoch: 1 [19200/60000] Loss: 0.129100
Train Epoch: 1 [25600/60000] Loss: 0.047927
Train Epoch: 1 [32000/60000] Loss: 0.039252
Train Epoch: 1 [38400/60000] Loss: 0.036285
Train Epoch: 1 [44800/60000] Loss: 0.011841
Train Epoch: 1 [51200/60000] Loss: 0.011470
Train Epoch: 1 [57600/60000] Loss: 0.060897

Test set: Average loss: 0.0000, Accuracy: 9848/10000 (98.48%)

Train Epoch: 2 [0/60000] Loss: 0.012237
Train Epoch: 2 [6400/60000] Loss: 0.053163
Train Epoch: 2 [12800/60000] Loss: 0.007935
Train Epoch: 2 [19200/60000] Loss: 0.023361
Train Epoch: 2 [25600/60000] Loss: 0.075113
Train Epoch: 2 [32000/60000] Loss: 0.010290
Train Epoch: 2 [38400/60000] Loss: 0.011733
Train Epoch: 2 [44800/60000] Loss: 0.133325
Train Epoch: 2 [51200/60000] Loss: 0.005644
Train Epoch: 2 [57600/60000] Loss: 0.012428

Test set: Average loss: 0.0000, Accuracy: 9861/10000 (98.61%)

Tr

In [8]:
model = CNN() 
model.load_state_dict(torch.load(MODEL_NAME))
model.eval()

traced_model = torch.jit.trace(model, torch.randn(1, 1, 28, 28))

traced_model.save("model_ts.pt")

2. **Конвертация в ONNX**
   - Экспортируйте обученную модель в формат **ONNX**.

In [1]:
! pip install onnx==1.13.1

You should consider upgrading via the 'F:\GitHub\HSE\dev_ops\hw1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [18]:
! pip install onnxruntime==1.14.1

You should consider upgrading via the 'F:\GitHub\HSE\dev_ops\hw1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
! pip uninstall numpy -y

Found existing installation: numpy 2.2.3
Uninstalling numpy-2.2.3:
  Successfully uninstalled numpy-2.2.3


In [3]:
! pip install numpy==1.26.4

You should consider upgrading via the 'F:\GitHub\HSE\dev_ops\hw1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
import torch.onnx

model = CNN() 
model.load_state_dict(torch.load(MODEL_NAME))
model.eval()

dummy_input = torch.randn(1, 1, 28, 28)  # (batch=1, channels=1, height=28, width=28)

onnx_filename = "mnist_cnn.onnx"
torch.onnx.export(
    model,  
    dummy_input,
    onnx_filename,
    input_names=["input"],
    output_names=["output"], 
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}, 
    opset_version=11
)

print(f"Модель успешно экспортирована в {onnx_filename}")


Модель успешно экспортирована в mnist_cnn.onnx


In [8]:
import onnx
import onnxruntime as ort
import numpy as np

onnx_model = onnx.load("mnist_cnn.onnx")
onnx.checker.check_model(onnx_model)  # Проверяем модель

ort_session = ort.InferenceSession("mnist_cnn.onnx")

x = np.random.randn(1, 1, 28, 28).astype(np.float32)
ort_inputs = {"input": x}
ort_outs = ort_session.run(["output"], ort_inputs)

print(f"Выход ONNX модели: {ort_outs}")

Выход ONNX модели: [array([[-23.250158 , -13.814761 , -10.290759 ,  -8.994146 , -19.839657 ,
         -8.587656 , -10.643445 , -10.049277 , -16.82718  , -14.6214905]],
      dtype=float32)]


3. **(Опционально) Конвертация в TensorRT (TRT)**
   - При необходимости, конвертируйте модель в формат **TensorRT** для повышения производительности инференса.

4. **Оптимизация модели средствами Torch/TensorFlow**
   - Примените встроенные методы оптимизации (например, quantization или pruning) для улучшения эффективности модели.

In [13]:
# model = CNN()
# model.load_state_dict(torch.load("mnist_cnn.pth"))
# model.eval()

# model.qconfig = torch.ao.quantization.get_default_qconfig('x86')
# model_fused = torch.ao.quantization.fuse_modules(model, [['conv1', 'relu1'], ['conv2', 'relu2']])
# model_prepared = torch.ao.quantization.prepare(model_fused)
# model_int8 = torch.ao.quantization.convert(model_prepared)

# torch.save(model_int8.state_dict(), "mnist_cnn_quantized.pth")


In [12]:
import torch

model = CNN()
model.load_state_dict(torch.load("mnist_cnn.pth"))
model.eval()

model.qconfig = torch.ao.quantization.get_default_qconfig('x86')
model_fused = torch.ao.quantization.fuse_modules(model, [['conv1', 'relu1'], ['conv2', 'relu2']])
model_prepared = torch.ao.quantization.prepare(model_fused)
model_int8 = torch.ao.quantization.convert(model_prepared)

scripted_model = torch.jit.script(model_int8)
scripted_model.save("mnist_cnn_quantized_ts.pt")  # TorchScript-модель


In [ ]:
import torch
import torch.ao.quantization as quantization

model = CNN()
model.load_state_dict(torch.load("mnist_cnn.pth"))
model.eval()

model.qconfig = quantization.QConfig(
    activation=quantization.default_observer,
    weight=quantization.default_weight_observer
)

model_fused = torch.ao.quantization.fuse_modules(model, [['conv1', 'relu1'], ['conv2', 'relu2']])
model_prepared = torch.ao.quantization.prepare(model_fused)
model_int8 = torch.ao.quantization.convert(model_prepared)

scripted_model = torch.jit.script(model_int8)
scripted_model.save("mnist_cnn_quantized_ts.pt")  # TorchScript-модель


5. **Оптимизация модели инструментами ONNX и (опционально) TRT**
   - Используйте оптимизирующие инструменты для ONNX (например, ONNX Runtime) для повышения производительности.
   - (Опционально) Оптимизируйте модель в формате TensorRT.

In [10]:
! pip3 install onnxoptimizer

You should consider upgrading via the 'F:\GitHub\HSE\dev_ops\hw1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
import onnx
import onnxoptimizer

onnx_model_path = "mnist_cnn.onnx"
onnx_model = onnx.load(onnx_model_path)

passes = [
    "eliminate_deadend",        # Убирает ненужные узлы
    "eliminate_identity",       # Убирает операции типа Identity
    "eliminate_nop_transpose",  # Убирает ненужные транспонирования
    "fuse_bn_into_conv",        # Сливает BatchNorm в Conv2d
    "fuse_add_bias_into_conv"   # Сливает Add в Conv2d
]

optimized_model = onnxoptimizer.optimize(onnx_model, passes)
optimized_onnx_path = "mnist_cnn_optimized.onnx"

onnx.save(optimized_model, optimized_onnx_path)

ort_session = ort.InferenceSession(optimized_onnx_path, providers=["CPUExecutionProvider"])

print("Доступные бэкенды:", ort.get_available_providers())

import numpy as np
x = np.random.randn(1, 1, 28, 28).astype(np.float32)  # MNIST вход

# Выполняем инференс
ort_inputs = {ort_session.get_inputs()[0].name: x}
ort_outs = ort_session.run(None, ort_inputs)

ort_outs

Доступные бэкенды: ['CPUExecutionProvider']


[array([[-15.927749 , -11.864381 ,  -7.2300553, -10.10805  , -13.666244 ,
         -10.109586 , -10.94195  , -12.203027 , -15.49283  , -13.321471 ]],
       dtype=float32)]

6. **Разработка микросервиса предобработки данных**
   - Спроектируйте микросервис для предобработки данных, необходимых для работы вашей модели.
   - Реализуйте сервис с использованием **Flask**, **FastAPI** или другого подходящего фреймворка.
   - Оформите микросервис в виде **Docker-контейнера**.

In [ ]:
! pip install fastapi uvicorn pillow

  Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)


You should consider upgrading via the 'F:\GitHub\HSE\dev_ops\hw1\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


: 

7. **Развёртывание моделей с помощью Triton Inference Server**
   - Запустите **Triton Inference Server**.
   - Задеплойте следующие версии модели:
     - Оригинальная модель.
     - Оптимизированная модель (на базе Torch/TensorFlow).
     - Модель в формате ONNX.
     - Оптимизированная модель ONNX.
     - (Опционально) Модель в формате TRT.

In [ ]:
! docker pull nvcr.io/nvidia/tritonserver:23.10-py3

In [ ]:
! docker run --gpus all -p 8000:8000 -p 8001:8001 -p 8002:8002 -v "F:/GITHUB/HSE/DEV_OPS/HW1/TRITON_INFERENCE_SERVER/models:/models" nvcr.io/nvidia/tritonserver:23.10-py3 tritonserver --model-repository=/models

8. **Настройка мониторинга метрик**
   - Настройте сбор и визуализацию метрик с помощью **Grafana** и **Prometheus**.
   - Обеспечьте мониторинг как для Triton Inference Server, так и для микросервиса предобработки.

In [27]:
! docker run -d -p 9090:9090 -v "F:\GitHub\HSE\dev_ops\hw1\prometheus\prometheus.yml:/etc/prometheus/prometheus.yml" prom/prometheus

7080efbc3e39b5cc57ec21f780b4b94ca9ce9a66c2e4e129e726144e9c5b02ae


http://host.docker.internal:9090

In [19]:
! docker run -d -p 3000:3000 --name grafana grafana/grafana


112e9de5f4e115e6d6f60581670cf7584666f1f5d6f63ca8b1b29cad2bab51d0


In [32]:
! pip install prometheus-fastapi-instrumentator

9. **Оркестрация сервисов с помощью docker-compose**
   - Поднимите весь стек сервисов через **docker-compose**, включающий:
     - Микросервис предобработки данных.
     - Triton Inference Server.
     - Систему мониторинга (Prometheus и Grafana).

10. **Тестирование и формирование отчёта**
    - Проведите тестирование всех версий модели, отправляя запросы на инференс.
    - Соберите логи работы сервисов.
    - Сформируйте итоговый файл с отчётом, включающим результаты тестирования, собранные логи и анализ производительности.